In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import glob

path = "Flight data/*_2016.csv"
frame=pd.DataFrame()
df_list=[]

for fname in glob.glob(path):    
    df=pd.read_csv(fname,index_col=None, header=0)
    df_list.append(df)
frame = pd.concat(df_list)



In [2]:
# pdata = pd.read_csv("September_2016.csv", sep=',')[["DAY_OF_WEEK","AIRLINE_ID", "ORIGIN_CITY_NAME", "DEST_CITY_NAME", "FL_NUM","DEP_TIME"]]
frame.head()
frame=frame.fillna(0)
frame["DEP_HOUR"]=frame["DEP_TIME"].astype(int)
frame["HOUR"]=(frame["DEP_HOUR"]//100)
frame.head()

,AIRLINE_ID,ARR_DELAY_NEW,ARR_TIME,CANCELLED,CARRIER,CARRIER_DELAY,DAY_OF_WEEK,DEP_DELAY_NEW,DEP_TIME,DEST,...,LATE_AIRCRAFT_DELAY,NAS_DELAY,ORIGIN,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,SECURITY_DELAY,Unnamed: 24,WEATHER_DELAY,DEP_HOUR,HOUR
0,20416,0.0,1857.0,0.0,NK,0.0,1,0.0,1524.0,MCO,...,0.0,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1524,15
1,20416,13.0,1924.0,0.0,NK,0.0,1,19.0,1552.0,MCO,...,0.0,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1552,15
2,20416,50.0,2010.0,0.0,NK,0.0,1,53.0,1635.0,MCO,...,50.0,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1635,16
3,20416,3.0,1923.0,0.0,NK,0.0,1,0.0,1539.0,MCO,...,0.0,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1539,15
4,20416,4.0,2112.0,0.0,NK,0.0,1,0.0,2003.0,FLL,...,0.0,0.0,MCO,13204,"Orlando, FL",0.0,0.0,0.0,2003,20


In [3]:
def f(row):
    if row == "1":
        val="MONDAY"
    elif row == "2":
        val="TUESDAY"
    elif row == "3":
        val="WEDNESDAY"
    elif row == "4":
        val="THURSDAY"
    elif row == "5":
        val="FRIDAY"
    elif row == "6":
        val="SATURDAY"
    else:
        val="SUNDAY"
    return val
frame['weekday_name'] = frame.apply(lambda x:f(str(x['DAY_OF_WEEK'])),axis=1)
frame.head()

,AIRLINE_ID,ARR_DELAY_NEW,ARR_TIME,CANCELLED,CARRIER,CARRIER_DELAY,DAY_OF_WEEK,DEP_DELAY_NEW,DEP_TIME,DEST,...,NAS_DELAY,ORIGIN,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,SECURITY_DELAY,Unnamed: 24,WEATHER_DELAY,DEP_HOUR,HOUR,weekday_name
0,20416,0.0,1857.0,0.0,NK,0.0,1,0.0,1524.0,MCO,...,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1524,15,MONDAY
1,20416,13.0,1924.0,0.0,NK,0.0,1,19.0,1552.0,MCO,...,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1552,15,MONDAY
2,20416,50.0,2010.0,0.0,NK,0.0,1,53.0,1635.0,MCO,...,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1635,16,MONDAY
3,20416,3.0,1923.0,0.0,NK,0.0,1,0.0,1539.0,MCO,...,0.0,MCI,13198,"Kansas City, MO",0.0,0.0,0.0,1539,15,MONDAY
4,20416,4.0,2112.0,0.0,NK,0.0,1,0.0,2003.0,FLL,...,0.0,MCO,13204,"Orlando, FL",0.0,0.0,0.0,2003,20,MONDAY


In [ ]:
df=DataFrame(frame,columns=["HOUR","weekday_name","ORIGIN_CITY_NAME","FL_NUM"])
df = DataFrame(df.groupby(['ORIGIN_CITY_NAME','weekday_name','HOUR'])["FL_NUM"].count())
df.head(n=25).reset_index()
df.head(n=25).to_csv("category.csv")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
city=input("text")
# weekday=input("text")

saturated=pd.read_csv("category.csv")
# saturated=saturated[saturated.weekday_name==weekday]
saturated=saturated[saturated.ORIGIN_CITY_NAME==city]
sns.factorplot(x="FL_NUM",y="HOUR",hue="weekday_name",data=saturated,saturation=2,size=10,aspect=1,kind="bar",legend=True)

plt.title('Number of flights',fontsize=15,y=1.08)
plt.xlabel('Number of flights')
plt.ylabel('Flights operated each hour')
ax = plt.gca()
# ax.get_xaxis().get_major_formatter().set_scientific(False)
plt.draw()
plt.figtext(1.0, 1.10, 'Date Generated:'+str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')), horizontalalignment='right')

fig1 = plt.gcf()
plt.show()
fig1.savefig("count_per_hour.png")